# Interpretable Clusters !

In [80]:
# libraries we're using
using CSV
using DataFrames
using CategoricalArrays

In [116]:
# load data
df = CSV.read("demo_clusters_final.csv", DataFrame)

# cluster is categorical variable
df.Cluster = categorical(df.Cluster)

# select all feature columns (numeric!)
df_numeric = df[:, 5:end-1]

print(df_numeric)

# set seed to get reproducible results
seed = 42

1361×53 DataFrame
  Row │ total population  prop males  prop females  prop of people under 5  prop of people aged 5-9  prop of people aged 10-14  prop of people aged 15-19  prop of people aged 20-24  prop of people aged 25-34  prop of people aged 35-44  prop of people aged 45-54  prop of people aged 55-59  prop of people aged 60-64  prop of people aged 65-74  prop of people aged 75-84  prop of people aged over 85  median aged  prop of white people  prop of black people  prop of native american people  prop of asian people(including indian and chinese)  prop of pacific islander  prop of employeed people 16 and over  prop of commuters who drive to work  prop of commuter who carpool to work  prop of commuter who took public transport to work  prop of commuters who walk to work  prop of commuters who work from home  prop with health insurance   mean commute time to work  num of home owners  number of homes  number of houses built 2020 or more recent  Number of houses built 2010 to 2019  nu

Excessive output truncated after 524288 bytes.

42

### Check different types of error - Which performs best

In [87]:
accuracy_tracker = []
loss_functions = ["misclassification", "gini", "entropy"]

for loss_fn in loss_functions

    lnr = IAI.OptimalTreeClassifier(
                    random_seed = seed,
                    criterion = loss_fn,
                    max_depth = 3,
                    minbucket = 5)
    
    grid = IAI.GridSearch(lnr,
        max_depth = 3,
        minbucket = 5
    )

    IAI.fit!(grid, df_numeric, df.Cluster)

    lnr = IAI.get_learner(grid)
    
    val_accuracy = IAI.score(lnr_depth, df_numeric, df.Cluster)
    
    # Add AUC score to AUC tracker
    append!(accuracy_tracker, val_accuracy)

    # adding tree visualization for interpretation
    IAI.show_in_browser(lnr)
end

In [88]:
println("--------AUC scores in Validation Set--------")
for i in range(1,length(loss_functions))
    println(loss_functions[i], ": ", round(accuracy_tracker[i], digits=4))
end

--------AUC scores in Validation Set--------
misclassification: 0.061
gini: 0.061
entropy: 0.061


Since we have similar performance across the splitting criteria loss functions, we can choose any. We use misclassification since it is the most straight-forward and our purposes don't require further granularity. 

### Check different depths - Which performs best?

In [100]:
# build classification tree

depths = [3, 4, 5]

grid = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed = seed,
        criterion=:misclassification,
        missingdatamode=:separate_class,
        minbucket = 5
    ),
    max_depth = depths,
    minbucket = 5,
)

# select all feature columns (numeric!)
df_selected = df[:, 4:end-1]

IAI.fit!(grid, df_numeric, df.Cluster)

All Grid Results:

 Row │ minbucket  max_depth  cp           train_score  valid_score  rank_valid_score
     │ Int64      Int64      Float64      Float64      Float64      Int64
─────┼───────────────────────────────────────────────────────────────────────────────
   1 │         5          3  0.00352609      0.821429     0.768949                 3
   2 │         5          4  0.000705219     0.896008     0.832763                 2
   3 │         5          5  0.00211566      0.933824     0.845477                 1

Best Params:
  minbucket => 5
  cp => 0.002115655853314524
  max_depth => 5

Best Model - Fitted OptimalTreeClassifier:
  1) Split: prop of people 25 years or older who have a grad or professional degree < 0.2722
    2) Split: prop of commuter who took public transport to work < 0.01743
      3) Split: total population < 41015
        4) Split: median montly housing costs < 1310
          5) Predict: 1 (94.44%), [4,68,0,0,0,0], 72 points, error 0.05556
          6) Split: number of houses built 1940 to 1949 < 599.6
            7) Predict: 0 (93.59%), [73,0,2,0,3,0], 78 points, error 0.0641
            8) Predict: 1 (55.56%), [0,5,2,1,1,0], 9 points, error 0.4444
        9) Split: prop of asian people(including indian and chinese) < 0.02125
          10) Predict: 1 (100.00%), [0,6,0,0,0,0], 6 points, error 0
          11) Predict: 0 (97.40%), [150,3,1,0,0,0], 154 points, error 0.02597
      12) Split: prop of black people < 0.7635
        13) Split: prop of people born in the US < 0.62
          14) Split: prop of naturalized citizens < 0.09362
            15) Predict: 1 (75.51%), [8,37,2,2,0,0], 49 points, error 0.2449
            16) Predict: 2 (89.33%), [2,6,134,2,0,6], 150 points, error 0.1067
          17) Split: prop of people 25 years or older who have a bachelors degree < 0.358
            18) Predict: 1 (94.55%), [0,208,3,2,0,7], 220 points, error 0.05455
            19) Predict: 3 (65.85%), [7,1,1,27,5,0], 41 points, error 0.3415
        20) Predict: 5 (98.06%), [0,1,1,0,0,101], 103 points, error 0.01942
    21) Split: prop of people aged 60-64 < 0.04711
      22) Split: median aged < 38.13
        23) Split: prop of people under 5 < 0.07972
          24) Predict: 3 (94.21%), [3,0,5,179,3,0], 190 points, error 0.05789
          25) Predict: 4 (60.00%), [1,0,1,0,3,0], 5 points, error 0.4
        26) Predict: 4 (100.00%), [0,0,0,0,10,0], 10 points, error 0
      27) Split: total population < 35268.2
        28) Split: prop of people aged 75-84 < 0.0308
          29) Predict: 3 (90.48%), [0,1,1,19,0,0], 21 points, error 0.09524
          30) Split: median household yearly income < 70570.7
            31) Predict: 1 (100.00%), [0,10,0,0,0,0], 10 points, error 0
            32) Predict: 4 (89.63%), [2,1,7,4,121,0], 135 points, error 0.1037
        33) Split: prop of commuters who drive to work < 0.7225
          34) Split: prop of people born in the US < 0.6456
            35) Predict: 2 (88.89%), [0,0,40,0,5,0], 45 points, error 0.1111
            36) Predict: 5 (45.45%), [1,0,0,1,4,5], 11 points, error 0.5455
          37) Predict: 0 (100.00%), [52,0,0,0,0,0], 52 points, error 0

In [101]:
lnr_depth = IAI.get_learner(grid)

# Evaluate the classification accuracy
accuracy = IAI.score(lnr_depth, df_numeric, df.Cluster)
println("Classification Accuracy: ", accuracy)
best_depth = lnr_depth.max_depth
println("Best tree depth: ", best_depth)

# Visualize the tree
IAI.show_in_browser(grid)

Classification Accuracy: 0.916972814107274
Best tree depth: 5


"/var/folders/p_/j5379bnd7t7f3l97gslk8q_80000gn/T/jl_2coujx.html"

Even though this is the best tree depth of the values we tested is 5, this is not super interpretable to distill into insights. Thus, we will explicitly print out trees for depth of 3 and 4 as well as see if those might be better at explaining the phenomena with acceptable accuracy scores (>80%).

#### Depth 4, Misclassification split

In [117]:
grid_4 = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed = seed,
        max_depth=4,             
        criterion=:misclassification, 
        missingdatamode=:separate_class 
    )
)

IAI.fit!(grid_4, df_numeric, df.Cluster)

All Grid Results:

 Row │ cp          train_score  valid_score  rank_valid_score
     │ Float64     Float64      Float64      Int64
─────┼────────────────────────────────────────────────────────
   1 │ 0.00208623     0.888772     0.821814                 1

Best Params:
  cp => 0.0020862308762169685

Best Model - Fitted OptimalTreeClassifier:
  1) Split: prop of commuter who took public transport to work < 0.0175
    2) Split: median household yearly income < 67040.8
      3) Split: number of homes < 21134.2
        4) Split: Number of houses built 2010 to 2019 < 2202.7
          5) Predict: 1 (100.00%), [0,68,0,0,0,0,0], 68 points, error 0
          6) Predict: 0 (100.00%), [7,0,0,0,0,0,0], 7 points, error 0
        7) Predict: 0 (100.00%), [33,0,0,0,0,0,0], 33 points, error 0
      8) Split: prop of commuters who drive to work < 0.7091
        9) Split: prop of people aged 45-54 < 0.1003
          10) Predict: 3 (100.00%), [0,0,0,31,0,0,0], 31 points, error 0
          11) Predict: 4 (83.67%), [5,1,1,0,41,0,1], 49 points, error 0.1633
        12) Split: total population < 25663.5
          13) Predict: 4 (43.75%), [1,10,0,7,14,0,0], 32 points, error 0.5625
          14) Predict: 0 (91.02%), [223,11,4,2,5,0,0], 245 points, error 0.0898
    15) Split: prop of poeple 25 years or older who have an associates degree < 0.07724
      16) Split: median aged < 36.05
        17) Split: prop of commuters who drive to work < 0.4349
          18) Predict: 6 (89.87%), [0,0,1,7,0,0,71], 79 points, error 0.1013
          19) Predict: 3 (89.68%), [1,5,0,113,4,0,3], 126 points, error 0.1032
        20) Split: prop of people aged 5-9 < 0.03703
          21) Predict: 6 (81.63%), [0,1,5,2,1,0,40], 49 points, error 0.1837
          22) Predict: 4 (72.60%), [3,1,12,1,53,0,3], 73 points, error 0.274
      23) Split: prop of naturalized citizens < 0.0941
        24) Split: prop of white people < 0.197
          25) Predict: 5 (100.00%), [0,0,0,0,0,99,0], 99 points, error 0
          26) Predict: 1 (80.20%), [12,235,8,18,7,12,1], 293 points, error 0.198
        27) Split: prop of people born in the US < 0.6313
          28) Predict: 2 (89.41%), [2,2,152,3,3,6,2], 170 points, error 0.1059
          29) Predict: 4 (85.71%), [0,1,0,0,6,0,0], 7 points, error 0.1429

In [118]:
lnr_depth_4 = IAI.get_learner(grid_4)

# Evaluate the classification accuracy
accuracy = IAI.score(lnr_depth_4, df_numeric, df.Cluster)
println("Classification Accuracy: ", accuracy)

IAI.show_in_browser(grid_4)

Classification Accuracy: 0.8714180749448934


"/var/folders/p_/j5379bnd7t7f3l97gslk8q_80000gn/T/jl_BYoc3o.html"

#### Depth 3, Misclassification split

In [119]:
grid_3 = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed = seed,
        max_depth=3,               
        criterion=:misclassification,  
        missingdatamode=:separate_class  
    )
)

IAI.fit!(grid_3, df_numeric, df.Cluster)

All Grid Results:

 Row │ cp          train_score  valid_score  rank_valid_score
     │ Float64     Float64      Float64      Int64
─────┼────────────────────────────────────────────────────────
   1 │ 0.00208623     0.796432     0.747304                 1

Best Params:
  cp => 0.0020862308762169823

Best Model - Fitted OptimalTreeClassifier:
  1) Split: prop of people 25 years or older who have a grad or professional degree < 0.2711
    2) Split: median montly housing costs < 1319.2
      3) Split: prop of white people < 0.2095
        4) Predict: 5 (98.97%), [0,1,0,0,0,96,0], 97 points, error 0.01031
        5) Predict: 1 (79.84%), [44,301,17,6,0,9,0], 377 points, error 0.2016
      6) Split: number of houses built 1990 to 1999 < 1741.5
        7) Predict: 2 (64.80%), [5,20,127,24,10,6,4], 196 points, error 0.352
        8) Predict: 0 (85.78%), [181,5,5,13,6,1,0], 211 points, error 0.1422
    9) Split: median aged < 36.05
      10) Split: prop of commuters who drive to work < 0.4349
        11) Predict: 6 (89.87%), [0,0,0,6,0,2,71], 79 points, error 0.1013
        12) Predict: 3 (88.97%), [2,1,4,129,6,0,3], 145 points, error 0.1103
      13) Split: total population < 35895.5
        14) Predict: 4 (69.74%), [0,7,6,6,106,0,27], 152 points, error 0.3026
        15) Predict: 0 (52.88%), [55,0,24,0,6,3,16], 104 points, error 0.4712

In [120]:
lnr_depth_3 = IAI.get_learner(grid_3)

# Evaluate the classification accuracy
accuracy = IAI.score(lnr_depth_3, df_numeric, df.Cluster)
println("Classification Accuracy: ", accuracy)

IAI.show_in_browser(grid_3)

Classification Accuracy: 0.7832476120499633


"/var/folders/p_/j5379bnd7t7f3l97gslk8q_80000gn/T/jl_K5n0Bi.html"

#### Interpretation with Tree of Depth 3
Cluster 1 - small population ---> less populated suburbs
Cluster 2 - large population, not taking public transit, educated ---> young professionals in the city
Cluster 3 - works from home, takes public transit, does not have high school degree ---> ????
Cluster 4 - large population, takes public transit, has high school degree ---> inner-city workers (????)
Cluster 5 - does not take public transit or work frm home, low college grads ---> ?????

#### Interpretation with Tree of Depth 4
Cluster 1 - small/medium population ---> less populated suburbs
Cluster 2 - large population, not walking to work, houses built in 1990-99 ---> wealthy millennials
Cluster 3 - large asian population that walk/not drive to work ---> chinatowns in various cities ?
Cluster 4 - large population, ethnic, educated (hs) ---> city
Cluster 5 - medium population, drive to work, very white areas --->